In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

#from mtflearn import ZPs

In [2]:
from mtflearn.features import autocorrelation, radial_profile, get_characteristic_length
from mtflearn.utils import normalize_image

In [3]:
filename = 'data\\monolayer_MoSe2_80K.npy'
img = np.load(filename)[0:1024, 0:1024]
img = normalize_image(img, 0, 1)

In [4]:
get_characteristic_length(img)

In [15]:
hh = autocorrelation(img)
ll = radial_profile(hh)

In [16]:
plt.plot(ll)
plt.axhline(ll.mean())

In [17]:
from scipy.signal import find_peaks

In [18]:
find_peaks(ll)

In [22]:
peaks, _ = find_peaks(ll, height=0)
plt.plot(peaks, ll[peaks], "x")
plt.plot(ll)
plt.axhline(ll.mean())